In [25]:
from pathlib import Path
from plip.structure.preparation import PDBComplex
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [26]:
HERE = Path(".").resolve()
DATA_DIR = HERE / "data"
PDB_DIR = HERE / "data_pdb"
PDB_DIR.mkdir(exist_ok=True)

In [27]:
#функция для конвертирования .cif. в pdb.:
import subprocess
import os

def convert_cif_to_pdb(cif_path, pdb_path):
    try:
        subprocess.run(
            ["obabel", str(cif_path), "-O", str(pdb_path)],
            check=True,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return True
    except Exception as e:
        print(f"Не удалось конвертировать {cif_path.name}: {e}")
        return False

In [28]:
#найдем все .cif файлы
cif_files = list(DATA_DIR.glob("*.cif"))
print(f"Найдено {len(cif_files)} .cif файлов")

Найдено 83 .cif файлов


In [29]:
# Конвертация
pdb_files = []
for cif in cif_files:
    pdb_name = cif.stem + ".pdb"
    pdb_path = PDB_DIR / pdb_name
    if convert_cif_to_pdb(cif, pdb_path):
        pdb_files.append(pdb_path)

print(f"Успешно конвертировано: {len(pdb_files)} файлов")

Успешно конвертировано: 83 файлов


In [32]:
from plip.structure.preparation import PDBComplex

pdb_file = pdb_files[0]

protlig = PDBComplex()
protlig.load_pdb(str(pdb_file))
protlig.analyze()

site = list(protlig.interaction_sets.values())[0]
print("Доступные атрибуты InteractionSet:")
print([attr for attr in dir(site) if not attr.startswith('_')])

Доступные атрибуты InteractionSet:
['Mapper', 'all_hbonds_ldon', 'all_hbonds_pdon', 'all_hydrophobic_contacts', 'all_itypes', 'all_pication_laro', 'all_pication_paro', 'altconf', 'bindingsite', 'find_unpaired_ligand', 'halogen_bonds', 'hbonds_ldon', 'hbonds_pdon', 'hydrophobic_contacts', 'interacting_chains', 'interacting_res', 'lig_members', 'ligand', 'metal_complexes', 'no_interactions', 'num_unpaired_hal', 'num_unpaired_hba', 'num_unpaired_hbd', 'output_path', 'pdbid', 'pication_laro', 'pication_paro', 'pistacking', 'refine_hbonds_ldon', 'refine_hbonds_pdon', 'refine_hydrophobic', 'refine_pication', 'refine_water_bridges', 'saltbridge_lneg', 'saltbridge_pneg', 'unpaired_hal', 'unpaired_hal_orig_idx', 'unpaired_hba', 'unpaired_hba_orig_idx', 'unpaired_hbd', 'unpaired_hbd_orig_idx', 'water_bridges']


In [33]:
#функция для анализа типов связей через PLIP
def get_interactions_from_pdb(pdb_path):
    protlig = PDBComplex()
    protlig.load_pdb(str(pdb_path))
    protlig.analyze()

    residue_interactions = {}

    for site in protlig.interaction_sets.values():
        interaction_map = {
            "hydrophobic": site.hydrophobic_contacts,
            "hbond": site.hbonds_ldon + site.hbonds_pdon,
            "waterbridge": site.water_bridges,
            "saltbridge": site.saltbridge_lneg + site.saltbridge_pneg,
            "pistacking": site.pistacking,
            "pication": site.pication_laro + site.pication_paro,
            "halogen": site.halogen_bonds,
            "metal": site.metal_complexes,
        }

        for itype, interactions in interaction_map.items():
            for i in interactions:
                res_key = f"{i.restype}_{i.reschain}:{i.resnr}"
                if res_key not in residue_interactions:
                    residue_interactions[res_key] = set()
                residue_interactions[res_key].add(itype)
    return residue_interactions

In [34]:
all_residues = set()
data = {}

for pdb_file in pdb_files:
    pdb_id = pdb_file.stem
    try:
        interactions = get_interactions_from_pdb(pdb_file)
        data[pdb_id] = interactions
        all_residues.update(interactions.keys())
    except Exception as e:
        print(f"    Пропущен: {e}")
        data[pdb_id] = {}

In [35]:
total_files = len(pdb_files)
processed_files = sum(1 for v in data.values() if v)
skipped_files = total_files - processed_files

print(f"Всего файлов: {total_files}")
print(f"Успешно обработано: {processed_files}")
print(f"Пропущено: {skipped_files}")

Всего файлов: 83
Успешно обработано: 83
Пропущено: 0


In [36]:
#формируем датасет
rows = []
for pdb_id in data:
    row = {"Protein": pdb_id}
    for res in sorted(all_residues):
        types = data[pdb_id].get(res, set())
        row[res] = ", ".join(sorted(types)) if types else 0
    rows.append(row)

df = pd.DataFrame(rows)
df.set_index("Protein", inplace=True)

In [38]:
output_file = HERE / "protein_ligand_interaction.csv"
df.to_csv(output_file)

df.head()

,ALA_A:130,ALA_A:157,ALA_A:158,ALA_A:167,ALA_A:172,ALA_A:177,ALA_A:178,ALA_A:207,ALA_A:211,ALA_A:228,...,VAL_D:51,VAL_D:57,VAL_D:63,VAL_D:67,VAL_D:83,VAL_D:85,VAL_D:87,VAL_D:91,VAL_D:93,VAL_D:94
Protein,,,,,,,,,,,,,,,,,,,,,
4RMZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4U9A,0,0,0,hbond,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4XS2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4Y73,0,hbond,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4YO6,0,0,0,0,0,0,0,0,0,0,...,hydrophobic,0,0,0,0,0,0,0,0,0
